In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data_GDP = pd.read_excel('GDP 1990-2020.xlsx')
data_Income_Outcome = pd.read_excel('Income and Outcome 2010-2020.xlsx')
data_Inflation = pd.read_excel('Inflation 1992-2020.xlsx')
data_Population = pd.read_excel('Population 2002-2020.xlsx')
data_Unemployment = pd.read_excel('Unemployment 2010-2021.xlsx')
data_2Inflation = data_Inflation[:]
Inflat_level = 100
for i in range(1992, 2022):
    Inflat_level *= data_Inflation[i]/100
    data_2Inflation[i] = Inflat_level
data_2Inflation['Unnamed: 0'] = 'Рівень інфляції'

In [ ]:
dataset = pd.concat([data_GDP, data_Population, data_Income_Outcome, data_Inflation, data_2Inflation, data_Unemployment], 
                                                                                    ignore_index = True)
dataset.pop('Unnamed: 1')
dataset.rename(columns = {'Unnamed: 0':int(0)}, inplace = True)
dataset.columns
for column in dataset.columns:
    dataset.rename(columns = {column:int(column)},inplace = True)
dataset.sort_index(axis = 1, inplace = True)
dataset.rename(columns = {0:"Type"}, inplace = True)
dataset.set_index('Type', inplace = True)
dataset

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20]
fig, axs = plt.subplots(2,3)
axs[0][0].plot(dataset.loc['ВВП'])
axs[0][0].set_title('ВВП')
axs[0][1].plot(dataset.loc['Рівень інфляції'])
axs[0][1].set_title('Рівень інфляції')
axs[1][0].plot(dataset.loc['Населення'])
axs[1][0].set_title('Населення')
axs[1][1].plot(dataset.loc['Інфляція'])
axs[1][1].set_title('Інфляція')
axs[0][2].plot(dataset.loc['Безробітне населення'])
axs[0][2].set_title('Безробітне населення')
axs[1][2].plot(dataset.loc['Доходи'])
axs[1][2].plot(dataset.loc['Витрати та заощадження'])
axs[1][2].legend(["Доходи","Витрати та заощадження"])
axs[1][2].set_title('Доходи та витрати')
plt.show()

In [ ]:
def pxy(dataset,rowx_name,rowy_name):
    rowx = dataset.loc[rowx_name,::]
    rowy = dataset.loc[rowy_name,::]
    rowx_notna = rowx.notna()
    rowy_notna = rowy.notna()
    rowx_mean = rowx.mean()
    rowy_mean = rowy.mean()
    Sx2 = 0
    S2 = 0
    cov = 0
    for i in range(len(rowx)):
        if rowx_notna[i]:
            Sx2 += pow(rowx[i]-rowx_mean,2)
        if rowy_notna[i]:
            S2 += pow(rowy[i]-rowy_mean,2)
        if rowy_notna[i] and rowx_notna[i]:
            cov += (rowx[i]-rowx_mean)*(rowy[i]-rowy_mean)
    return cov/np.sqrt(Sx2*S2)

dataset5 = dataset.loc[::,2015:2020]
dataset10 = dataset.loc[::,2010:2020]
print('Для 2015-2020')
print(f'pxy Рівень споживчих цін/Доходи і витрати населення : {pxy(dataset5,"Рівень інфляції","Доходи")}')
print(f'pxy Індекс споживчих цін/Безробіття : {pxy(dataset5,"Інфляція","Безробітне населення")}')
print(f'pxy ВВП/Населення : {pxy(dataset5,"ВВП","Населення")}')
print('Для 2010-2020')
print(f'pxy Рівень споживчих цін/Доходи і витрати населення : {pxy(dataset10,"Рівень інфляції","Доходи")}')
print(f'pxy Індекс споживчих цін/Безробіття : {pxy(dataset10,"Інфляція","Безробітне населення")}')
print(f'pxy ВВП/Населення : {pxy(dataset10,"ВВП","Населення")}')

In [ ]:
def regression(plot, dataset, rowx_name,rowy_name):
    rowx = dataset.loc[rowx_name,::]
    rowy = dataset.loc[rowy_name,::]
    plot.set_title(rowy_name + " to " + rowx_name + f'   pxy = {pxy(dataset,rowx_name,rowy_name)}')
    plot.set_xlabel(rowx_name)
    plot.set_ylabel(rowy_name)
    plot.scatter(rowx, rowy)
    m, b = np.polyfit(rowx, rowy, 1)
    plot. plot(rowx, m*rowx + b)
    plot.legend([f'{m}x'+(f'+{b}' if b>0 else f'{b}')])

plt.rcParams['figure.figsize'] = [30, 20]
fig, axs = plt.subplots(3,2)
regression(axs[0][0],dataset5,"Інфляція","Доходи")
regression(axs[1][0],dataset5,"Інфляція","Безробітне населення")
regression(axs[2][0],dataset5,"Інфляція","ВВП")
regression(axs[0][1],dataset10,"Інфляція","Доходи")
regression(axs[1][1],dataset10,"Інфляція","Безробітне населення")
regression(axs[2][1],dataset10,"Інфляція","ВВП")

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20]
fig, axs = plt.subplots(3,2)
regression(axs[0][0],dataset5,"Інфляція","ВВП")
regression(axs[1][0],dataset5,"Безробітне населення","ВВП")
regression(axs[2][0],dataset5,"Доходи","ВВП")
regression(axs[0][1],dataset10,"Інфляція","ВВП")
regression(axs[1][1],dataset10,"Безробітне населення","ВВП")
regression(axs[2][1],dataset10,"Доходи","ВВП")